<a href="https://colab.research.google.com/github/nvrancovich/gh22-twitter-sentiment-analysis/blob/main/polaridad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importacíon

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

In [ ]:
url = 'https://drive.google.com/file/d/1IklQ716OMB-ZYETa4NZM6Eq7cJnrDzOK/view?usp=share_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url, index_col=0)

# Limpieza

## La hora está adelantada 3 horas respecto a la de Argentina

In [ ]:
df.Date = pd.to_datetime(df.Date)
df.Date[0]

Timestamp('2022-12-08 11:03:54+0000', tz='UTC')

In [ ]:
df.Date = df.Date - pd.Timedelta(hours=3)
df.Date[0]

Timestamp('2022-12-08 08:03:54+0000', tz='UTC')

## Removiendo todos los tweets que contienen a más de un participante. Sería muy complicado procesar comparaciones y asignar valores de polaridad a cada uno de los participantes en estos tweets.

In [ ]:
def filtro(tweet):
  contador = 0

  participantes = {
    'marcos':['marcos','marquitos','primo'],
    'tomas':['tomás','holder','tomas'],
    'julieta':['julieta','juli'],
    'coti':['coti','coty'],
    'daniela':['daniela','dani','pestañela'],
    'nacho':['nacho'],
    'agustin':['agustin','agustín','frodo','agus'],
    'maxi':['maxi'],
    'alexis':['alexis','cone','conejo'],
    'walter':['walter','alfa'],
    'romina':['romina','romi'],
    'thiago':['thiago'],
    'maria':['maria','cata'],
    'mora':['mora'],
    'lucila':['lucila','tora'],
    'juan':['juan'],
    'juliana':['juliana','tini']
  }

  for participante in participantes:
    for apodo in participantes[participante]:
      if apodo in tweet:
        contador += 1
        resultado = participante
        break
  if contador == 1:
    return resultado
  else:
    return False

In [ ]:
df['Tweet'] = df['Tweet'].str.lower()
df['Participante'] = df['Tweet'].apply(filtro)

In [ ]:
df.drop(df[df.Participante == False].index, inplace=True)

## Removiendo todos los tweets que contienen links, estos no contienen opininones o son demasiado dificiles de procesar (son tweets que acompañan multimedia y sin contexto no son útiles)

In [ ]:
df[df.Tweet.str.contains('https')].sample(5)

,Date,Tweet,Participante
19849,2022-12-07 22:21:44+00:00,tomatela alexis. encima que sos igual a una toruga ninja jaja #gh22 https://t.co/rfe7zwrkqn,alexis
746,2022-12-08 03:51:11+00:00,"aparte no se iba a caer y podía pasar bien, no somos tontos julieta #gh22 #gh2022 https://t.co/mjmeifzbu3",julieta
55717,2022-12-05 22:32:54+00:00,agradezco que daniela no se fue todos queriamos verlas en la compra semanal #gh2022 #gh22 https://t.co/tmrum56bv1,daniela
46853,2022-12-06 11:59:58+00:00,"es un psicópata, busca hacerla sentir incómoda. para cuando alguien que vaya a gritar ""agustín sos el próximo""??? quiero que caiga de la nube de pedo en la que vive pensando que todos afuera bancan todo lo que hace #gh22 #gh2022 https://t.co/y8nhcm7kqw",agustin
55596,2022-12-05 22:34:56+00:00,nadie voto a cata - nadie habla del fraude\n#granhermano #granhermano2022 #gh2022 #gh22 #fraudoneta #granfraude2022 #granfraude22 https://t.co/tbjq1lxbob,maria


In [ ]:
df.drop(df[df.Tweet.str.contains('https')].index, inplace=True)

In [ ]:
df.head(5)

,Date,Tweet,Participante
1,2022-12-08 08:03:18+00:00,bueno romina yo te voto a vos #gh22,romina
13,2022-12-08 07:59:15+00:00,ojalá se vaya romina .\nno la soporto hace tres semanas.\n#gh22 \nbuenos dias.,romina
15,2022-12-08 07:58:54+00:00,se imaginan si otra vez la producción mete mano y lo sacan a marcos? colapsan los hospitales. #gh22 #granhermanoargentina,marcos
28,2022-12-08 07:49:34+00:00,@rucciaantonella @mariaogong @eeemiliano señora. la producción elige quien se queda y quién se va de #granhermano #gh2022 #gh22. \nquedó demostrado el domingo pasado.\nsaludos. \nfraude en gh.,maria
39,2022-12-08 07:47:11+00:00,@iszgrl @santi_cruzzz @cbastianberdejo @gh2022ok julieta es súper carismática! y la gente la quiere jajaja \nhay que tener una mezcla entre ser buen jugador y tener carisma y que la gente te quiera jaja \n\n#gh22 #gh2022 #granhermano,julieta


## Armando una función que remueva las menciones y los '#'

In [ ]:
import re

def limpiar_tweet(tweet):
  clean_tweet = re.sub("@[A-Za-z0-9_]+","", tweet)
  clean_tweet = re.sub("#[A-Za-z0-9_]+","", clean_tweet)
  clean_tweet = clean_tweet.replace('\n\n',' ')
  clean_tweet = clean_tweet.replace('\n','')
  
  return clean_tweet

In [ ]:
df['Tweet'] = df['Tweet'].apply(limpiar_tweet)

In [ ]:
df.sample(5)

,Date,Tweet,Participante
53326,2022-12-05 23:01:10+00:00,maxi es demasiado agresivo,maxi
24915,2022-12-07 21:52:19+00:00,"se ríe de un compañero, no espero menos de agustín",agustin
17982,2022-12-07 22:28:26+00:00,quiere estar en placa agustin jajajajajjakaja cagaste anda armando tu valija pelotudo q nadie te va a apoyar,agustin
97623,2022-12-04 22:50:15+00:00,cata callate la puta madre,maria
34018,2022-12-06 23:57:27+00:00,aaa julieta hizo la espontánea que reina,julieta


# Analisis

## Haciendo análisis de polaridad utilizando la librería pysentimiento




In [ ]:
!pip install pysentimiento

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet
analyzer = create_analyzer(task="sentiment", lang="es")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--pysentimiento--robertuito-sentiment-analysis/snapshots/e3be95c8efad7f480ce8aab2221188ecb78e40f3/config.json
Model config RobertaConfig {
  "_name_or_path": "pysentimiento/robertuito-sentiment-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEG",
    "1": "NEU",
    "2": "POS"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEG": 0,
    "NEU": 1,
    "POS": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_

In [ ]:
df['Tweet'] = df['Tweet'].apply(preprocess_tweet )

In [ ]:
df.sample(5)

,Date,Tweet,Participante
42028,2022-12-06 21:58:28+00:00,jsjsjsjaja por si a alguno ahí en la casa le faltaba enterarse que marcos es campeon nacional ahí santiago se encargó de que se enteren emoji cara lanzando un beso emoji,marcos
33026,2022-12-07 01:19:22+00:00,juan se cree que va a volver a la casa después de que lo echaron con el 88 por ciento,juan
50535,2022-12-06 00:03:42+00:00,"habrá un día en el que alfa diga ""amo tal cosa"" en vez de tanto ""no soporto""? no soporta nada! pero todos tienen que soportarlo a él",walter
54865,2022-12-05 22:44:34+00:00,lo que banco a romi,romina
1729,2022-12-08 02:01:19+00:00,todos los que se llenan la boca hablando de alfa comiendo de sus pizzas,walter


In [ ]:
def polaridad(tweet):
  return analyzer.predict("tweet")

In [ ]:
muestra = df.sample(10)
muestra['Polaridad'] = muestra['Tweet'].apply(polaridad)

In [ ]:
muestra

,Date,Tweet,Participante,Polaridad
81439,2022-12-04 23:49:47+00:00,literalmente sigue siendo tendencia nacho al 9009fraude en gh,nacho,"AnalyzerOutput(output=NEU, probas={NEU: 0.762, POS: 0.129, NEG: 0.109})"
42683,2022-12-06 21:20:36+00:00,pueden hacer un informe sobre el mal gasto que hacen en la casa? sobre todo thiago con el agua,thiago,"AnalyzerOutput(output=NEU, probas={NEU: 0.762, POS: 0.129, NEG: 0.109})"
42281,2022-12-06 21:53:04+00:00,"yo no lo banco a alfa, pero en parte me da pena porque las pruebas están hechas para los más jóvenes, nunca le pongan un juego de adivinanzas eh",walter,"AnalyzerOutput(output=NEU, probas={NEU: 0.762, POS: 0.129, NEG: 0.109})"
41085,2022-12-06 22:17:04+00:00,vamos nacho lpm,nacho,"AnalyzerOutput(output=NEU, probas={NEU: 0.762, POS: 0.129, NEG: 0.109})"
39560,2022-12-06 22:41:57+00:00,banco que quieran sacar a alfa,walter,"AnalyzerOutput(output=NEU, probas={NEU: 0.762, POS: 0.129, NEG: 0.109})"
89557,2022-12-04 23:09:37+00:00,??? la produccion diciendo que viene pareja la votacion para que gastemos plata. si nadie votona cata,maria,"AnalyzerOutput(output=NEU, probas={NEU: 0.762, POS: 0.129, NEG: 0.109})"
59523,2022-12-05 22:10:15+00:00,"a alfa cuando le tiran la posta en la cara lo único que dice es no no es así, yo no dije eso y arranca a subir el tono",walter,"AnalyzerOutput(output=NEU, probas={NEU: 0.762, POS: 0.129, NEG: 0.109})"
15316,2022-12-07 22:41:12+00:00,ay nacho con el ojo rojo por las pestañas pobre jaja,nacho,"AnalyzerOutput(output=NEU, probas={NEU: 0.762, POS: 0.129, NEG: 0.109})"
98323,2022-12-04 22:47:06+00:00,quiere llorar el viejo pobre .hay gente q todavía no entiende este nuevo publico .nacho al 9009 !!!!,nacho,"AnalyzerOutput(output=NEU, probas={NEU: 0.762, POS: 0.129, NEG: 0.109})"
64675,2022-12-05 15:07:18+00:00,daniela está re arriba desde anoche o es cosa mía?,daniela,"AnalyzerOutput(output=NEU, probas={NEU: 0.762, POS: 0.129, NEG: 0.109})"


Esta librería no nos da ningun resultado

## Haciendo análisis de polaridad utilizando la librería sentiment-analysis-spanish

In [ ]:
!pip install sentiment-analysis-spanish

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sentiment_analysis_spanish import sentiment_analysis
sentiment = sentiment_analysis.SentimentAnalysisSpanish()

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
def polaridad(tweet):
  return round(float(sentiment.sentiment(tweet)),4)

In [ ]:
muestra = df[:5000]
muestra['Polaridad'] = muestra['Tweet'].apply(polaridad)
muestra.sample(20)

,Date,Tweet,Participante,Polaridad
72470,2022-12-05 00:32:12+00:00,cortaron a alfa forreando al que el cree que hizo expontanea para cuidarlo,walter,0.0298
32892,2022-12-07 01:31:08+00:00,"el tape de agustin siendo eliminado de gran hermano, se vuelve realidad ???",agustin,0.4168
97373,2022-12-04 22:51:17+00:00,marcos es todo lo que está bien,marcos,0.7299
71608,2022-12-05 00:51:17+00:00,a quien va a votar marcos ahora akdksjd piquete,marcos,0.0500
98116,2022-12-04 22:47:48+00:00,"pobres chicos, vaya a saber hasta cuándo van a tener que aguantar a ése vendepatria. cómo se nota que la gente que vota no tiene que aguantarlo. ojalá te vayas solo alfa. sos insoportable",walter,0.0000
37119,2022-12-06 23:09:07+00:00,a veces no soporto a daniela,daniela,0.0964
83088,2022-12-04 23:32:21+00:00,no entiendo q le ven a nacho para q lo sigan salvando,nacho,0.0000
80131,2022-12-04 23:54:40+00:00,pestañela la próxima sos vos,daniela,0.8750
59919,2022-12-05 22:08:44+00:00,maxi pelado soberbio del orto ándate con tu noviecita al cabify,maxi,0.1870
3897,2022-12-08 00:15:46+00:00,"bueno ahora sí,hay que fletar a agustin che. quiero ver su cara,si es que el villero no lo salva",agustin,0.0000


In [ ]:
muestra.groupby('Participante').describe()

Polaridad                                                      \
                 count      mean       std  min     25%      50%       75%   
Participante                                                                 
agustin         7680.0  0.097964  0.167787  0.0  0.0001  0.00680  0.123200   
alexis           489.0  0.154113  0.219869  0.0  0.0005  0.03080  0.262900   
coti            1572.0  0.147320  0.201754  0.0  0.0007  0.03360  0.276300   
daniela         4060.0  0.100928  0.183025  0.0  0.0001  0.00650  0.115400   
juan             107.0  0.083569  0.181344  0.0  0.0000  0.00070  0.053750   
juliana           29.0  0.128741  0.216463  0.0  0.0004  0.01550  0.204700   
julieta         2813.0  0.145751  0.217202  0.0  0.0001  0.01680  0.249600   
lucila            72.0  0.102707  0.192373  0.0  0.0000  0.00110  0.114550   
marcos          4378.0  0.121533  0.211101  0.0  0.0001  0.00825  0.147525   
maria           2312.0  0.108410  0.197853  0.0  0.0001  0.00470  0.119350   
maxi             461.0  0.123638  0.194786  0.0  0.0002  0.01880  0.160700   
mora              85.0  0.055062  0.115440  0.0  0.0000  0.00010  0.050600   
nacho           2365.0  0.082756  0.156793  0.0  0.0001  0.00610  0.079900   
romina          1758.0  0.167547  0.224973  0.0  0.0007  0.04005  0.293900   
thiago          1152.0  0.086737  0.158531  0.0  0.0001  0.00580  0.088675   
tomas             98.0  0.109511  0.210195  0.0  0.0000  0.00130  0.125625   
walter          3293.0  0.092308  0.180462  0.0  0.0000  0.00260  0.087700   

                      
                 max  
Participante          
agustin       0.9993  
alexis        0.9971  
coti          0.9955  
daniela       1.0000  
juan          0.8894  
juliana       0.9499  
julieta       0.9990  
lucila        0.9870  
marcos        1.0000  
maria         0.9991  
maxi          0.9950  
mora          0.5264  
nacho         1.0000  
romina        0.9983  
thiago        0.9829  
tomas         0.9678  
walter        0.9960

Si bien esta librería nos da resultados, está muy lejos de representar las realidades de popularidad.